In [6]:
"""Наши данные представляют собой четыре таблицы:

ratings1 и ratings2 — таблицы с данными о выставленных пользователями оценках фильмов. 
Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы 
с оценками фильмов.
userId — уникальный идентификатор пользователя, который выставил оценку;
movieId — уникальный идентификатор фильма;
rating — рейтинг фильма.

dates — таблица с датами выставления всех оценок.
date — дата и время выставления оценки фильму.

movies — таблица с информацией о фильмах.
movieId — уникальный идентификатор фильма;
title — название фильма и год его выхода;
genres — жанры фильма."""
import pandas as pd
movies = pd.read_csv('data_test/movies.csv')
#Сколько уникальных фильмов представлено в таблице movies?
movies['movieId'].nunique()
#Сколько уникальных пользователей в таблице ratings1?
ratings1_df = pd.read_csv('data_test/ratings1.csv')
ratings2_df = pd.read_csv('data_test/ratings2.csv')
ratings1_df['userId'].nunique()
#В каком году было выставлено больше всего оценок?...используйте таблицу dates.
dates = pd.read_csv('data_test/dates.csv')
pd.to_datetime(dates['date']).dt.year

#years_dates = dates.dt.year
#print (years_dates)

0         2000
1         2000
2         2000
3         2000
4         2000
          ... 
100831    2017
100832    2017
100833    2017
100834    2017
100835    2017
Name: date, Length: 100836, dtype: int64

In [7]:
"""Итак, давайте склеим  ratings1 и ratings2 по строкам, так как они имеют одинаковую структуру 
столбцов. Для этого передадим их списком в функцию concat(). Помним, что параметр axis по умолчанию 
равен 0, объединение происходит по строкам, поэтому не трогаем его. 
Примечание. Обратите внимание, что concat является функцией библиотеки, а не методом DataFrame.
Поэтому её вызов осуществляется как pd.concat(...)."""
ratings = pd.concat([ratings1_df,ratings2_df])
ratings
#по умолчанию concat сохраняет первоначальные индексы объединяемых таблиц, а обе наши таблицы индексировались,
#начиная от 0. Чтобы создать новые индексы, нужно выставить параметр ignore_index на True:
ratings = pd.concat([ratings1_df,ratings2_df],ignore_index=True)
ratings
#Казалось бы, совсем другое дело! Но это ещё не всё. Давайте узнаем количество строк в таблицах
#ratings и dates, ведь нам предстоит вертикально склеить их между собой:
print ('Number of strings in table "ratings":',ratings.shape)
print ('Number of strings in table "dates":',dates.shape)
print (ratings.shape[0] == dates.shape[0])
"""Pазмерность таблиц разная — как такое могло произойти?
На самом деле очень просто: при выгрузке данных информация об оценках какого-то пользователя попала
в обе таблицы (ratings1 и ratings2). В результате конкатенации случилось дублирование строк. 
В данном примере их легко найти — выведем последнюю строку таблицы ratings1 и первую строку 
таблицы ratings2:"""
ratings1_df.tail(1)
ratings2_df.head(1)
"""Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(),
который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей,
выставив параметр ignore_index в методе drop_duplicates() на значение True:"""
ratings = ratings.drop_duplicates(ignore_index=True)
print('Number of rows in table"ratings":',ratings.shape[0])
#Наконец, мы можем добавить к нашей таблице с оценками даты их выставления.
#Для этого конкатенируем таблицы ratings и dates по столбцам:
ratings_dates = pd.concat([ratings,dates], axis=1)
ratings_dates.tail(7)


Number of strings in table "ratings": (100837, 3)
Number of strings in table "dates": (100836, 1)
False
Number of rows in table"ratings": 100836


,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


In [14]:
"""Допустим, в ваше распоряжение предоставлена директория "./Root/users/". В данной директории 
содержатся csv-файлы, в каждом из которых хранится информация об идентификаторах пользователей 
(user_id) и ссылки на их фотографии (photo_url). Каждый файл из папки users имеет примерно 
следующую структуру:
Img
При проверке в директории может быть сколько угодно файлов (директория может изменяться в 
зависимости от устройства файловой системы).
Вам необходимо написать функцию concat_user_files(path), параметром которой является path — путь 
до директории. Функция должна объединить информацию из предоставленных вам файлов в один DataFrame
и вернуть его. 
Список названий всех файлов, находящихся в директории, вы можете получить с помощью функции 
os.listdir(path) из модуля os (модуль уже импортирован в файле main.py). Например, для директории
"./Root/users/" результатом работы функции будет список:
print(os.listdir('./Root/users/'))
['users2.csv', 'users1.csv', 'users3.csv']
Примечание. Модуль os позволяет работать с операционной системой компьютера прямо из кода.
Подробнее о нем вы можете почитать здесь.
Отсортируйте этот список, прежде чем производить объединение файлов.
Когда вы получите отсортированный список, вам останется только прочитать все csv-файлы из списка 
в цикле и объединить прочитанные таблицы между собой.
Однако обратите внимание, что метод os.listdir() возвращает только список имён файлов в указанной
директории, а при чтении файла необходимо указывать полный путь до него. То есть путь для чтения
будет таким:
'./Root/users/{file_name}'
Не забудьте обновить индексы результирующей таблицы после объединения.
Учтите, что на тестовом наборе файлов в результате объединения могут возникнуть дубликаты, 
от которых необходимо будет избавиться."""
import os
print(os.listdir('./Root/users/'))
import pandas as pd
"""Вам необходимо написать функцию concat_user_files(path), параметром которой является path 
- путь до директории. Функция должна объединить информацию из предоставленных вам файлов в 
один DataFrame и вернуть его. Не забудьте обновить индексы результирующей таблицы после объединения.
Учтите тот момент, что в результате объединения могут возникнуть дубликаты, от которых необходимо
будет избавиться. """
    
def concat_users_files(path):
    data = pd.DataFrame()
    file_names = os.listdir(path)
    file_names.sort()
    for file in file_names:
        tmp_data = pd.read_csv(path + '/' + file)
        data = pd.concat([data, tmp_data], axis=0, ignore_index=True)
    data = data.drop_duplicates()
    return data

if __name__ == '__main__':
    data = concat_users_files('./Root/users/')

['users1.csv', 'users3.csv', 'users2.csv']
                          image_url user_id
0  http://example.com/img/id001.png   id001
1  http://example.com/img/id002.jpg   id002
2  http://example.com/img/id003.bmp   id003
3  http://example.com/img/id004.jpg   id004
4  http://example.com/img/id005.png   id005
5  http://example.com/img/id006.png   id006
6  http://example.com/img/id007.png   id007
7  http://example.com/img/id008.png   id008

[8 rows x 2 columns]

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: './Root/users/'

In [56]:
"""Основные параметры метода join()

other — таблица, которую мы присоединяем. При объединении она является «правой», а исходная 
таблица, от имени которой вызывается метод, является «левой».
how — параметр типа объединения. Он может принимать значения 'inner', 'left' (left outer), 'right'
(right outer), и 'outer' (full outer). По умолчанию параметр установлен на 'left'.
on — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение
по индексам из «правой».
lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах."""
joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='left'
)
joined_false
"""Обратите внимание, что в данном случае у нас получилось два столбца, соответствующих 
идентификатору фильма: один — из «левой» таблицы (movieId), а другой — из «правой» (movieId_right).
Однако это не тот результат, который мы хотели, ведь мы не получили соответствия фильмов и их 
рейтингов. Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо 
использовать ключевой столбец в «правой» таблице в качестве индекса. Это можно сделать с помощью 
метода set_index(). Также необходимо указать название ключа в параметре on."""
joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
joined.head()

"""Аналогично предыдущему, метод merge() предназначен для слияния двух таблиц по ключевым столбцам
или по индексам. Однако, в отличие от join(), метод merge() предлагает более гибкий способ
управления объединением, благодаря чему является более популярным.
Основные параметры метода merge():
right — присоединяемая таблица. По умолчанию она является «правой».
how — параметр типа объединения. По умолчанию принимает значение 'inner'.
on — параметр, который определяет, по какому столбцу происходит объединение. Определяется 
автоматически, но рекомендуется указывать вручную.
left_on — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр 
отвечает за наименования ключевого столбца исходной таблицы.
right_on — аналогично предыдущему, параметр отвечает за наименование ключевого столбца 
присоединяемой таблицы.
Метод merge() в первую очередь предназначен для слияния таблиц по заданным ключам, поэтому он не 
требует установки ключевых столбцов в качестве индекса присоединяемой таблицы. Кроме того, 
данный метод позволяет объединять даже таблицы с разноимёнными ключами. Таким образом, merge() 
проще в использовании и более многофункционален, чем схожие методы."""
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
merged.head()
#Проверим, что число строк в таблице ratings_dates совпадает с числом строк в результирующей
# таблице merged:
'Numbers of rows in table"ratings_dates:"',ratings_dates.shape[0]
'Numbers of rows in table"merged:"',merged.shape[0]
ratings_dates.shape[0] == merged.shape[0]
#Возникает вопрос: почему мы выбрали тип объединения left, а не full, например?
#Найти ответ нам поможет пример. Объединим ratings_dates с movies по ключевому столбцу movieId,
#но с параметром how='outer' (full outer) и выведем размер таблицы, а также её «хвост»:
merged2 = ratings_dates.merge(
    movies,
    on='movieId',
    how='outer'
)
'Number of rows in table "merged2":',merged2.shape[0]
merged2.tail()
"""Оказывается, в таблице movies содержались фильмы, которым ещё не были выставлены оценки.
В результате объединения типом full outer информация о фильмах перенеслась из таблицы movies в
результирующую таблицу. Однако, поскольку оценки фильмам ещё не были выставлены, соответствующие
столбцы таблицы ratings_dates заполнились пропусками(NaN).Такие фильмы были записаны в конец таблицы
Метод merge() с внешним (outer) типом объединения может использоваться как аналог метода concat()
при объединении таблиц с одинаковой структурой (одинаковые количество и названия столбцов) по
строкам. В таком случае все одноимённые столбцы таблиц будут считаться ключевыми."""
#Рассмотрим пример: объединим таблицы ratings1 и ratings2, как мы уже делали раньше, 
# но теперь используем метод merge():
merge_ratings = ratings1_df.merge(ratings2_df, how='outer')
'Number of raws in table "merge_ratings:"', merge_ratings.shape[0]
merge_ratings
"""Обратите внимание, что при использовании метода merge() для склейки двух таблиц у нас
автоматически пропали дубликаты, которые мы видели при использовании метода concat(). Это
особенность метода merge() — автоматическое удаление дублей."""
#Дано две исходных таблицы:
data_1 = pd.DataFrame({'Value':[100,45,80],
                      'Group':[1,4,5]},
                      index=['I0','I1','I2'])
data_1
data_2 = pd.DataFrame({'Company': ['Google', 'Amazon', 'Facebook'],
                       "Add": ['S0', 'S1', 'S7']},
                      index = ['I0','I1', 'I3'])
data_2
data_joined_1 = data_1.join(data_2, how='inner')
data_joined_2 = data_1.join(data_2, how='outer')
data_joined_3 = data_1.join(data_2, how='left')
data_joined_4 = data_1.join(data_2, how='right')
data_joined_1

a = pd.DataFrame({'A': ['a','b','c'],'B': [103, 214, 124], 'C': [1, 4, 2]})
b = pd.DataFrame({'V': ['d', 'b', 'c'], 'U': [1393.7, 9382.2, 1904.5], 'C': [1, 3, 2]})
ex_1 = a.join(b, how='inner', rsuffix='_r')
ex_2 = a.merge(b, how='left', on='C')
ex_3 = b.join(a.set_index('C'), how='right', on='C')
ex_4 = a.merge(b, how='right', on='C')
ex_5 = a.merge(b, how='inner', on='C')
ex_3

#ЗАДАНИЕ 7.5 
#Даны две таблицы: items_df, в которой содержится информация о наличии товаров на складе,
#и purchase_df — с данными о покупках товаров.
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})
purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})
"""Информация в таблицах представлена в виде следующих столбцов:
item_id — идентификатор модели;
vendor — производитель модели;
stock_count — имеющееся на складе количество данных моделей (в штуках);
purchase_id — идентификатор покупки;
price — стоимость модели в покупке."""
#Сформируйте DataFrame merged, в котором в результате объединения purchase_df и items_df останутся
#модели, которые учтены на складе и имели продажи.
#Из таблицы merged найдите суммарную выручку, которую можно было бы получить от продажи всех товаров,
#которые учтены на складе и имели продажи. Результат занесите в переменную income.

df_merged = items_df.merge(purchase_df, how='inner', on='item_id')
df_merged
income = sum (df_merged['stock_count'] * df_merged['price'])#.sum()
income

19729490